In [53]:
## Import necessary modules
import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee

sys.path.append(os.path.abspath('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/entwuerfe/xls_testruns/'))
from ce_funclib import determine_kernzeit as dtkz
from ce_funclib import continuity_check

#%matplotlib inline
%matplotlib tk


## Import data frome pickle generated from muß ein file mit agentenstats sein
arcpth='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/'


In [54]:
######## GET A LIST OF MATCHING .xls FILES FROM THE GIVEN DIRECTORY

In [55]:
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('CE_al'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

xlfilelist=collectxlfiles(arcpth)
#xlfilelist
#examplefile=xlfilelist[233]

In [56]:
###### TEST FOR DATA IN FILE, SORT OUT EMPTY FILES 

In [57]:
## good dataframes do per definition not contain any zero values
## fill bad DFs with nan?

def filetoframe(exfile):
    exframe=pd.read_excel(exfile) # this is a regular pd.DataFrame
    datecell=exframe.iloc[0,1]
    sheet_datetime=pd.to_datetime(datecell,format='%d.%m %Y : %H')
    sheet_date=sheet_datetime.date()
    
    integritycheck=exframe.iloc[2,1] # files with data have "agenten" here, files with no calls have a 'nan'

    if integritycheck != 'Agenten':
        # if it's empty, keep date for filling it later
        print('Exception: ', end='')
        except_status='ex'
        
        usefulcols={0:'tstamp',1:'agent',3:'an',4:'be',22:'vl',24:'ht_float',29:'tt_float'} # map cols to decent names
        exframe=exframe.reindex(columns=sorted(usefulcols.keys()))
        exframe.rename(columns=usefulcols,inplace=True)        
        exframe=exframe[0:1] # strip text rows and the mangled sum row
        print(sheet_datetime)
        
        exframe['tstamp']=sheet_datetime
        exframe['date']=sheet_date
        exframe['agent']='platzhalter'
        exframe[['wd','ww','mm','yy']]=exframe['tstamp'].dt.strftime('%a,%W,%m,%Y').str.split(',',expand=True) # make ww,yy,mm,wd columns
        exframe['bz']=exframe['tstamp'].apply(dtkz)
        exframe['ort']=exframe['agent'].str[0] # split the identifier into useable columns
        exframe['id']='foobar' # split the identifier into useable columns
        
        # integers should be of appropriate datatype, we received them as strings
        # exframe[['vl','an','be','ww','mm','yy']]=exframe[['vl','an','be','ww','mm','yy']].astype(np.int64) #just for the beauty of it
        exframe[['ww','mm','yy']]=exframe[['ww','mm','yy']].astype(np.int64) #just for the beauty of it
        # exframe.fillna(0, inplace=True) 
        return exframe,except_status
        
    else:
        except_status='reg'
        
        exframe.columns=range(0,30) # rename columns to a temporarily more readable format, fancy rename later
        usefulcols={0:'tstamp',1:'agent',3:'an',4:'be',22:'vl',24:'ht_float',29:'tt_float'} # map cols to decent names
        exframe=exframe[sorted(usefulcols.keys())] # skip cols and keep the ones we need
        exframe.rename(columns=usefulcols,inplace=True) # rename cols
        exframe=exframe[3:-1] # strip text rows and the mangled sum row
        exframe['tstamp']=pd.to_datetime(exframe['tstamp'],format=' %d.%m.%Y %H:%M ')
        exframe['date']=exframe['tstamp'].dt.date
        exframe[['wd','ww','mm','yy']]=exframe['tstamp'].dt.strftime('%a,%W,%m,%Y').str.split(',',expand=True) # make ww,yy,mm,wd columns
        exframe['bz']=exframe['tstamp'].apply(dtkz)
        
        exframe['ort']=exframe['agent'].str[0] # split the identifier into useable columns
        exframe['id']=exframe['agent'].str[-6:] # split the identifier into useable columns
        exframe['agent']=exframe['agent'].str[2:-7] # split the identifier into useable columns
        
        # integers should be of appropriate datatype, we received them as strings
        exframe[['vl','an','be','ww','mm','yy']]=exframe[['vl','an','be','ww','mm','yy']].astype(np.int64) #just for the beauty of it

        return exframe,except_status

In [58]:
example_badframe,badstatus=filetoframe('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-04-17.xls')
example_goodframe,goodstatus=filetoframe('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-05-17.xls')
example_badframe

Exception: 2017-04-17 00:00:00


,tstamp,agent,an,be,vl,ht_float,tt_float,date,wd,ww,mm,yy,bz,ort,id
0,2017-04-17,platzhalter,NaN,NaN,NaN,NaN,NaN,2017-04-17,Mon,16,4,2017,n,p,foobar


In [59]:
example_goodframe.head(3)

,tstamp,agent,an,be,vl,ht_float,tt_float,date,wd,ww,mm,yy,bz,ort,id
3,2017-05-17 07:00:00,pletaan,1,1,0,0.8333,0.35,2017-05-17,Wed,20,5,2017,n,B,335334
4,2017-05-17 13:00:00,pletaan,1,1,0,8.4833,7.4833,2017-05-17,Wed,20,5,2017,k,B,335334
5,2017-05-17 15:00:00,beckeca,1,1,0,2.4333,2.3333,2017-05-17,Wed,20,5,2017,k,H,428869


In [60]:
framelist=list()
exceptionlist=list()
for xfile in xlfilelist:
    
    #print('file:',xfile)
    frame_from_file,except_status=filetoframe(xfile)
    #print(frame_from_file.columns)
    #print(frame_from_file['date'])
    if except_status=='ex':
        exceptionlist.append(xfile)
    framelist.append(frame_from_file)

exceptionlist

Exception: 2017-04-17 00:00:00
Exception: 2017-05-14 00:00:00


['/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-04-17.xls',
 '/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-05-14.xls']

In [61]:
#### produce a unified frame with all data and sort it by timstamp and agentname
bigframeii=pd.concat(framelist)

bigframeii.sort_values(['tstamp','agent'],inplace=True)
bigframeii.reset_index(drop=True,inplace=True) # there you go

In [62]:
# bigframeii
# die exklusivlogins müssen zusammengelegt werden
unify_id={'gesinst':'995887','stanzju':'878457','papkeda':'891914'}
bigframeii.loc[bigframeii['id'] == unify_id['gesinst'],'agent'] = 'gesinst'
bigframeii.loc[bigframeii['id'] == unify_id['stanzju'],'agent'] = 'stanzju'
bigframeii.loc[bigframeii['id'] == unify_id['papkeda'],'agent'] = 'papkeda'

In [63]:
### some date locator play, can conveniently be checked against a single xls file
#def check_single_day(day):
#    dayvalues=bigframeii.loc[bigframeii['date'] == day]
#    print('htsum',dayvalues['ht_float'].sum(), end=', ')
#    print('bearbeitete sum',dayvalues['be'].sum())
#check_single_day(datetime.date(2017,4,17)) # shows that days wihtout calls are in the frame, too
#check_single_day(datetime.date(2017,4,18)) # shows that days wihtout calls are in the frame, too

In [64]:
#### get all dates and check whether they're contiguous
datenserie_uniq=bigframeii['date'].unique().tolist()
tage_bestand=len(datenserie_uniq)
tage_start=datenserie_uniq[0]
tage_ende=datenserie_uniq[-1:]

missing_dates=continuity_check(datenserie_uniq)
if not missing_dates:
    print('no dates are missing')
else:
    print('the following dates are not within the frame:')
    print(missing_dates)

no dates are missing


In [65]:
### PARSE AGENT DATA
### What I want:
### * get a list of all agents that have worked in the period 
### * get data for each agent
### * get average of all agents as a reference
### per agent:
### ** get all calls that have lasted longer than x times the average of all agents
### ** get a plot of all calls (by timestamp)
### ** get a plot of all days (by date)
### ** get their tendencies over the weeks (? vacation dates missing and so on)

# bigframeii.tail(10)

In [66]:
# get all agents available and create frames for kern and neben
allagents_list=sorted(bigframeii['agent'].unique())
allagents_list.extend(['Hagenow','Berlin','Alle'])
standorte=bigframeii.ort.unique().tolist()

bigk=bigframeii.loc[bigframeii['bz']=='k']
bign=bigframeii.loc[bigframeii['bz']=='n']

**we can't figure out individual calls anyway, since raw data calls have been grouped by hours already  
so we can go on and group by days to figure out averages**

In [83]:
def group_and_add_average(agentname,frame,gruppierung):
    # step one: filter by agent; if agent is a location-bound group, filter by location and change agent name to group name
    if agentname == 'Hagenow':
        nur_agent=frame.loc[frame['ort']=='H'].copy()
        nur_agent['agent']='Hagenow'
        nur_agent['id']='000001'
    elif agentname == 'Berlin':
        nur_agent=frame.loc[frame['ort']=='B'].copy()
        nur_agent['agent']='Berlin'
        nur_agent['id']='000002'
    elif agentname == 'Alle':
        #nur_agent=frame.loc[frame['ort'].isin(standorte)].copy()
        nur_agent=frame.copy()
        nur_agent['agent']='Alle'
        nur_agent['id']='000000'    
    else:
        nur_agent=frame.loc[frame['agent']==agentname]
    
    
    
    
    
    # step 2: split into kern and neben
    k=nur_agent.loc[nur_agent['bz']=='k']
    if k.empty:
        print()
        print(agentname,end=' ')
        print('keine Kernzeit group_and_add_average')
        print('###')
        
    n=nur_agent.loc[nur_agent['bz']=='n']
    if n.empty:
        print()
        print(agentname,end=' ')
        print('keine Nebenzeit group_and_add_average')
        print('###')

    # step 3: group by day (instead of hour, as it is now) and add average ht,tt
    def group_and_average(agframe):
        ### ttstamp is dropped and date will be the new index; all others summed or reduced
        colfx_day={'agent':'first','an':'sum','be':'sum','vl':'sum','ht_float':'sum','tt_float':'sum','wd':'first','ww':'first', 'mm':'first','yy':'first','bz':'first','ort':'first','id':'first'}
        ### ttstamp is dropped, date is dropped and ww will be the new index; all others summed or reduced
        colfx_week={'agent':'first','an':'sum','be':'sum','vl':'sum','ht_float':'sum','tt_float':'sum','wd':'first','mm':'first','yy':'first','bz':'first','ort':'first','id':'first'}
        
        if gruppierung=='tag':
            grpd=agframe.groupby('date').agg(colfx_day)
        elif gruppierung=='woche':
            grpd=agframe.groupby('ww').agg(colfx_week)
        elif gruppierung=='nursplit':
            grpd=agframe.copy()
        
        grpd['aht']=grpd['ht_float']/grpd['be']
        grpd['att']=grpd['tt_float']/grpd['be']
        grpd['acw']=grpd['aht']-grpd['att']

        return grpd

    # step 4 get stats grouped by day and with the average column
    k_agent=group_and_average(k)
    n_agent=group_and_average(n)

    return k_agent,n_agent

**dictionary of frames**

In [84]:
### generate frames grouped by day and by week for every agent, put them in a dictionary
zeiten={}
print('collecting and grouping times (neben, kern) for')
for namen in allagents_list:
    kern_byday,neben_byday=group_and_add_average(namen,bigframeii,'tag')
    kern_byweek,neben_byweek=group_and_add_average(namen,bigframeii,'woche')
    
    zeiten[namen]={'k_day':kern_byday,'k_week':kern_byweek,'n_day':neben_byday,'n_week':neben_byweek}

collecting and grouping times (neben, kern) for

haustst keine Nebenzeit group_and_add_average
###

haustst keine Nebenzeit group_and_add_average
###

jakobir keine Kernzeit group_and_add_average
###

jakobir keine Kernzeit group_and_add_average
###

platzhalter keine Kernzeit group_and_add_average
###

platzhalter keine Kernzeit group_and_add_average
###

reismat keine Nebenzeit group_and_add_average
###

reismat keine Nebenzeit group_and_add_average
###

steffci keine Nebenzeit group_and_add_average
###

steffci keine Nebenzeit group_and_add_average
###


In [ ]:
zeiten['haustst']['n_day']

In [ ]:
def decminutes_to_mmss(decimal):
    #print(decimal)
    tdelta=timedelta(minutes=decimal)
    sekunden=tdelta.seconds
    minuten=(sekunden % 3600) // 60
    restsekunden=str(sekunden %60).zfill(2)
    mmssstring='{}:{}'.format(minuten, restsekunden)
    return mmssstring

def maptix2labels(ticks):
    ylabelz=list()
    for tic in ticks:
        #print(tic)
        tic=abs(tic)
        sstr=decminutes_to_mmss(tic)
        ylabelz.append(sstr)
    return ylabelz

In [ ]:
# colors
bgkern='#FFF7F2'
bgnebn='#F8FFF2'
aht="#21a9ff"
att="#ceecff"
aac="#c4c4c4"
zielzeit="#FF006E"
bars="#A06A00"
aav='#000C00'

In [ ]:
def plotit(agent,ww_or_dd):
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(17,7))
    
    ### preliminary deduction from parameters
    if ww_or_dd.lower() == 'woche':
        kzeit=zeiten[agent]['k_week'].copy()
        nzeit=zeiten[agent]['n_week'].copy()
    elif ww_or_dd.lower() == 'tage':
        kzeit=zeiten[agent]['k_day'].copy()
        nzeit=zeiten[agent]['n_day'].copy()
        
    ### check empty frames
    if (kzeit.empty and nzeit.empty):
        print('ueberhaupt keine Calls')
    elif kzeit.empty:
        print('keine calls in der Kernzeit')
        kzeit=kzeit.reindex(nzeit.index)#.fillna(0)
    elif nzeit.empty:
        print('keine calls in der Nebenzeit')
        nzeit=nzeit.reindex(kzeit.index)#.fillna(0)


    kmax=(kzeit['aht'].max())+0.5
    nmax=(nzeit['aht'].max())+0.5
    commonmax=max(kmax,nmax)
    commonmin=-0.25
    
    ersterZeitpunkt=min(min(kzeit.index),min(nzeit.index))
    letzterZeitpunkt=max(kzeit.index[-1],nzeit.index[-1])
    StartStr=str(ersterZeitpunkt)
    EndeStr=str(letzterZeitpunkt)
    
    calls_zeitraum_k=kzeit['be'].sum()
    calls_zeitraum_n=nzeit['be'].sum()
    

    htmean_k=kzeit['aht'].replace(0,np.NaN).mean()
    if np.isnan(htmean_k):
        print('this is nan')
        htmean_k=0
    htmean_n=nzeit['aht'].replace(0,np.NaN).mean() # decent mean value without the zeroes jan-mar
    if np.isnan(htmean_n):
        print('this is nan')
        htmean_n=0
    av_all_k=zeiten['Alle']['k_week']['aht'].replace(0,np.NaN).mean()
    av_all_n=zeiten['Alle']['n_week']['aht'].replace(0,np.NaN).mean()

    ### plots

    ax3 = ax1.twinx()
    ax3.tick_params('y', labelsize=6, labelcolor=bars)

    ax4 = ax2.twinx()
    ax4.tick_params('y', labelsize=6, labelcolor=bars)

    kcalls=ax3.bar(kzeit.index, kzeit['be'], width=0.7, alpha=0.1, color=bars, label='calls')
    ncalls=ax4.bar(nzeit.index, nzeit['be'], width=0.7, alpha=0.1, color=bars, label='calls')

    kaht,=ax1.plot(kzeit.index,kzeit['aht'],color=aht,label="aht")
    katt,=ax1.plot(kzeit.index,kzeit['att'],color=att,label="att")
    kacw,=ax1.plot(kzeit.index,kzeit['acw'],color=aac,label="acw")
    naht,=ax2.plot(nzeit.index,nzeit['aht'],color=aht,label="aht")
    natt,=ax2.plot(nzeit.index,nzeit['att'],color=att,label="att")
    nacw,=ax2.plot(nzeit.index,nzeit['acw'],color=aac,label="acw")

    kziel=ax1.axhline(y=3.5,color=zielzeit,ls=':',alpha=0.75, label='3:30 min')
    kreal=ax1.axhline(y=htmean_k,color=aht,ls='--',alpha=0.9, label=str(decminutes_to_mmss(htmean_k)))
    kalle=ax1.axhline(y=av_all_k,color=aav,ls='-.',alpha=0.2, label=str(decminutes_to_mmss(av_all_k)))
    nziel=ax2.axhline(y=1.5,color=zielzeit,ls=':',alpha=0.75, label='1:30 min')
    nreal=ax2.axhline(y=htmean_n,color=aht,ls='--',alpha=0.9, label=str(decminutes_to_mmss(htmean_n)))
    nalle=ax2.axhline(y=av_all_n,color=aav,ls='-.',alpha=0.2, label=str(decminutes_to_mmss(av_all_n)))

    ### ax1 labels
    ax1.set_ylim(commonmin,commonmax)

    minloc=AutoMinorLocator(4)
    ax1.yaxis.set_minor_locator(minloc)
    ax1.yaxis.set_minor_formatter(ScalarFormatter()) # is the same as major formatter

    left_tix_mj=ax1.get_yticks()
    left_tix_mn=ax1.get_yticks(minor=True)
    left_lbl_mj=maptix2labels(left_tix_mj)
    left_lbl_mn=maptix2labels(left_tix_mn)

    ax1.yaxis.set_ticklabels(left_lbl_mj)
    ax1.yaxis.set_ticklabels(left_lbl_mn,minor=True,size=6)


    ### ax2 labels
    ax2.set_ylim(ax1.get_ylim())

    ax2.yaxis.set_minor_locator(minloc)
    ax2.yaxis.set_minor_formatter(ScalarFormatter()) # is the same as major formatter

    left_tix_mj=ax2.get_yticks()
    left_tix_mn=ax2.get_yticks(minor=True)
    left_lbl_mj=maptix2labels(left_tix_mj)
    left_lbl_mn=maptix2labels(left_tix_mn)

    ax2.yaxis.set_ticklabels(left_lbl_mj)
    ax2.yaxis.set_ticklabels(left_lbl_mn,minor=True,size=6)

    ### color adjustments, titles, legend
    ax1.set_facecolor(bgkern)
    ax2.set_facecolor(bgnebn)

    desc_k,desc_n=str(int(calls_zeitraum_k)),str(int(calls_zeitraum_n))
    ax1.set_title('Kernzeit'+' Calls gesamt: '+desc_k, size=9)
    ax2.set_title('Nebenzeit'+' Calls gesamt: '+desc_n, size=9)

    ax1.set_xlabel(ww_or_dd, size=7)
    ax2.set_xlabel(ww_or_dd, size=7)
    ax1.tick_params('x', labelsize=8)
    ax2.tick_params('x', labelsize=8)

    ax1.set_ylabel('Minuten', rotation=90)
    ax4.set_ylabel('Calls',rotation=90,color=bars)

    f.suptitle('Bearbeitungszeiten '+agent+' nach '+ww_or_dd+' ab März 2017 bis '+ww_or_dd+' '+EndeStr)

    f.legend((kaht,katt,kacw,kziel,kreal,kalle,kcalls),('handling','talk','afterwork','zielzeit','Øzeit agent','Øzeit alle','calls'),fontsize=7,ncol=2,loc='upper right',borderaxespad=2)

    
    ### Abspeichern
    heute=datetime.date.today().strftime('%Y_%m_%d')
    bild_filename=str(heute+'_'+agent+'_'+ww_or_dd+'_'+StartStr+'-'+EndeStr)
    savepath='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/ce_teamleitung/plots/agenten_und_standorte/'
    speichernin=os.path.join(savepath,bild_filename)
    print(speichernin)
    #f.savefig(speichernin,ext='png')
    #plt.close()

In [ ]:
for person in allagents_list[10:11]:
    print(person)
    plotit(person,'Woche')


In [ ]:
#### isin function is pretty neat thing for filtering
#### obviously, ww is another datatype than mm, normalization required!
zeiten['gesinst']['n_day'].loc[zeiten['gesinst']['n_day']['ww'].isin([32,33,34,35,36,37,38,39,40,41])]
